In [2]:
import basic_system_data
import geopandas as gpd
import pyproj
from pylab import *
import math

#must use overpass API to get larger boxes of data
# wget http://api.openstreetmap.org/api/0.6/map?bbox=-1.5494,53.2767,-1.3053,53.3159 -O ~/data/osm/dcc.osm
#ogr2ogr --config OSM_USE_CUSTOM_INDEXING NO -skipfailures -f "ESRI Shapefile" ~/data/osm/dcc.shp/ ~/data/osm/dcc.osm

#deprecated as shapefile is missing some sites
#def importBluetoothSitesFromShapeFile():
#    df_sites = gpd.GeoDataFrame.from_file('/home/charles/data/msc/dcc/examples/BluetoothUnits.shp') #in bng
#    for i in range(0, len(df_sites.index)):
#        if math.isnan(df_sites['C2_No'].iloc[i]):
#            continue
#        sensorID = str(int(df_sites['C2_No'].iloc[i]))
#        easting = df_sites['Easting'].iloc[i]
#        northing = df_sites['Northing'].iloc[i]
#        sensorType = "bluetooth"
#        sql="INSERT INTO sensorsite (sensorID, sensorType, easting, northing)  VALUES ( '%s', '%s', %f, %f);"%(sensorID, sensorType, easting, northing)
#       print sql
#       cur.execute(sql)
#       con.commit()

def importBluetoothSitesFromWebCsv():
    #NB i mnaually removed commas from the name field in the csv download file first
    fn = "/home/charles/data/msc/dcc/web_bluetooth_sites.csv"
    c=0
    for line in open(fn):
        c+=1
        if c<3:
            continue
        line=line.strip()
        fields=line.split(",")
        (name, sensorID, desc, grid, ori) =fields
        if sensorID[0:3]=="MAC":
            sensorID = sensorID[7:]
        easting  = float(grid[0:6])
        northing = float(grid[6:12])
        sensorType = "bluetooth"
        sql="INSERT INTO sensorsite (sensorID, sensorType, easting, northing)  VALUES ( '%s', '%s', %f, %f);"%(sensorID, sensorType, easting, northing)
        print sql
        cur.execute(sql)
        con.commit()

def importRouteDefintions():
    fn = "/home/charles/data/msc/dcc/sample2_JourneyTimes20161205_201612WithRouteGuide_sheet2.csv"
    for line in open(fn):
        line=line.strip()
        fields=line.split(",")
        (foo,routeID,bar,ack,origin,destination,fez)=fields
        origin=origin[0:5]
        destination=destination[0:5]
        sql="INSERT INTO route (routeID, originSiteID, destSiteID)  VALUES ( '%s', '%s', '%s');"%(routeID, origin, destination)
        print sql
        cur.execute(sql)
        con.commit()

def importRouteCounts():
    fn = "/home/charles/data/msc/dcc/sample2_JourneyTimes20161205_201612WithRouteGuide_sheet1.csv"
    for line in open(fn):
        line=line.strip()
        fields=line.split(",")
        (foo,routeID,timestamp,window,count,bar)=fields
        window=int(window)
        count=int(count)
        sql="INSERT INTO routecount (routeID, timestamp, winlenseconds, count)  VALUES ( '%s', '%s', %d, %d);"%(routeID, timestamp, window, count)
        print sql
        cur.execute(sql)
        con.commit()

con = psycopg2.connect(database='TDSmain', user='aseem_awad')
cur = con.cursor()
wgs84  = pyproj.Proj(init='epsg:4326')  #WGS84
bng    = pyproj.Proj(init='epsg:27700') #british national grid

importBluetoothSitesFromWebCsv()
importRouteDefintions()
importRouteCounts()


OperationalError: FATAL:  role "charles" does not exist
